In [ ]:
!pip uninstall opencv-python -y
!pip install opencv-python-headless

In [ ]:
import os
import sys

import cv2
import numpy as np
from PIL import Image
from tqdm import tqdm

sys.path.append(os.path.join("..", ".."))
from utils.s3_bucket import S3Bucket

In [ ]:
bucket = S3Bucket("ava-cv-raw-photo-bucket")
input_prefix = "Pl@ntNet/plants/images"
output_prefix = "Pl@ntNet/plants-processed/images"

In [ ]:
def contrast_enhancement(np_image):
    """
    Increases contrast of an image using the LAB color space method.
    Copied from https://stackoverflow.com/a/41075028
    """
    # converting to LAB color space
    lab = cv2.cvtColor(np_image, cv2.COLOR_BGR2LAB)
    l_channel, a, b = cv2.split(lab)

    # Applying CLAHE to L-channel
    # feel free to try different values for the limit and grid size:
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    cl = clahe.apply(l_channel)

    # merge the CLAHE enhanced L-channel with the a and b channel
    limg = cv2.merge((cl,a,b))

    # Converting image from LAB Color model to BGR color spcae
    return cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)

In [ ]:
def smooth(np_image):
    return cv2.bilateralFilter(
    src=np_image,
    d=11,
    sigmaColor=60,
    sigmaSpace=40,
    borderType=cv2.BORDER_DEFAULT
)

In [ ]:
for obj in tqdm(bucket.filter(prefix=input_prefix)):
    input_key = obj.key
    if not input_key.endswith((".jpg", ".jpeg", ".png")):
        continue
        
    image = bucket[obj.key]
    np_image = np.array(image)
    enhanced_np_image = contrast_enhancement(np_image)
    smoothed_np_image = smooth(enhanced_np_image)
    
    output_key = input_key.replace(input_prefix, output_prefix)
    bucket[output_key] = Image.fromarray(smoothed_np_image, "RGB")